<a href="https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/cert_prof_time_series/class_03/TS%20-%20W3%20-%2014%20-%20Atividade_Avaliativa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

adaptado de [Certificado Profissional Desenvolvedor do TensorFlow](https://www.coursera.org/professional-certificates/tensorflow-in-practice) de [Laurence Moroney](https://laurencemoroney.com/)

# Uso de RNNs para prever séries temporais

Bem-vindo! Na atividade avaliativa anterior, você usou uma rede neural profunda vanilla para criar previsões para séries temporais geradas.

Desta vez, você usará as camadas do Tensorflow para processar dados de sequência, como as camadas Recurrent ou LSTMs, para ver como essas duas abordagens se comparam.

Vamos começar!

In [ ]:
# Baixar arquivos adicionais para o laboratório
!wget https://github.com/fabiobento/dnn-course-2024-1/raw/main/00_course_folder/cert_prof_time_series/class_03/saved_model_images.zip
!unzip -n -q saved_model_images.zip

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass

## Gerando os dados

A próxima célula inclui um conjunto de funções auxiliares para gerar e plotar a série temporal:

In [ ]:
def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Tempo")
    plt.ylabel("Valor")
    plt.grid(False)

def trend(time, slope=0):
    return slope * time

def seasonal_pattern(season_time):
    """Apenas um padrão arbitrário, você pode alterá-lo se desejar"""
    return np.where(season_time < 0.1,
                    np.cos(season_time * 6 * np.pi),
                    2 / np.exp(9 * season_time))

def seasonality(time, period, amplitude=1, phase=0):
    """Repete o mesmo padrão em cada período"""
    season_time = ((time + phase) % period) / period
    return amplitude * seasonal_pattern(season_time)

def noise(time, noise_level=1, seed=None):
    rnd = np.random.RandomState(seed)
    return rnd.randn(len(time)) * noise_level

Você gerará os mesmos dados de série temporal sintéticos dos laboratórios passados.

**Observe que, desta vez, toda a geração é feita em uma função e as variáveis globais são salvas em uma classe de dados. Isso é feito para evitar o uso do escopo global, como foi feito anteriormente no curso.**

Se você nunca usou classes de dados antes, elas são apenas classes Python que fornecem uma sintaxe conveniente para armazenar dados. Você pode ler mais sobre elas em [docs](https://docs.python.org/3/library/dataclasses.html).



In [ ]:
def generate_time_series():
    # A dimensão de tempo ou a coordenada x da série temporal
    time = np.arange(4 * 365 + 1, dtype="float32")

    # A série inicial é apenas uma linha reta com uma interceptação y
    y_intercept = 10
    slope = 0.005
    series = trend(time, slope) + y_intercept

    # Adição de sazonalidade
    amplitude = 50
    series += seasonality(time, period=365, amplitude=amplitude)

    # Adicionando algum ruído
    noise_level = 3
    series += noise(time, noise_level, seed=51)
    
    return time, series


# Salve todas as variáveis "globais" na classe G (G significa global)
@dataclass
class G:
    TIME, SERIES = generate_time_series()
    SPLIT_TIME = 1100
    WINDOW_SIZE = 20
    BATCH_SIZE = 32
    SHUFFLE_BUFFER_SIZE = 1000
    

# Plotar a série gerada
plt.figure(figsize=(10, 6))
plot_series(G.TIME, G.SERIES)
plt.show()

## Processamento dos dados

Como você já codificou as funções `train_val_split` e `windowed_dataset` durante as atividades anteriores, desta vez elas são fornecidas para você:

In [ ]:
def train_val_split(time, series, time_step=G.SPLIT_TIME):

    time_train = time[:time_step]
    series_train = series[:time_step]
    time_valid = time[time_step:]
    series_valid = series[time_step:]

    return time_train, series_train, time_valid, series_valid


# Dividir o conjunto de dados
time_train, series_train, time_valid, series_valid = train_val_split(G.TIME, G.SERIES)

In [ ]:
def windowed_dataset(series, window_size=G.WINDOW_SIZE, batch_size=G.BATCH_SIZE, shuffle_buffer=G.SHUFFLE_BUFFER_SIZE):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.shuffle(shuffle_buffer)
    dataset = dataset.map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

# Aplicar a transformação ao conjunto de treinamento
dataset = windowed_dataset(series_train)

## Definição da arquitetura do modelo

Agora que você tem uma função que processará os dados antes que eles sejam inseridos na rede neural para treinamento, é hora de definir a arquitetura das camadas. Diferentemente das semanas ou cursos anteriores, nos quais você define as camadas e compila o modelo na mesma função, aqui você precisará primeiro concluir a função `create_uncompiled_model` abaixo. 

Isso é feito para que você possa reutilizar as camadas do seu modelo para o ajuste da taxa de aprendizado e o treinamento real.

Dica:
- Preencha as camadas `Lambda` no início e no final da rede com as funções lamda corretas.
- Você deve usar `SimpleRNN` ou `Bidirectional(LSTM)` como camadas intermediárias.
- A última camada da rede (antes da última `Lambda`) deve ser uma camada `Dense`.

In [ ]:
def create_uncompiled_model():

    ### INICIE SEU CÓDIGO AQUI
    
    model = tf.keras.models.Sequential([ 
        tf.keras.layers.Lambda(),
        
        tf.keras.layers.Lambda()
    ]) 
    
    ### TERMINE SEU CÓDIGO AQUI

    return model

In [ ]:
# Teste seu modelo não compilado
uncompiled_model = create_uncompiled_model()

try:
    uncompiled_model.predict(dataset)
except:
    print("Sua arquitetura atual é incompatível com o conjunto de dados em janela, tente ajustá-la.")
else:
    print("Sua arquitetura atual é compatível com o conjunto de dados com janelas!)")

## Ajuste da taxa de aprendizado

Como você viu na aula, é possível aproveitar os _callbacks_ do Tensorflow para variar dinamicamente a taxa de aprendizagem durante o treinamento. Isso pode ser útil para ter uma noção melhor de qual taxa de aprendizado se adapta melhor ao problema em questão.

**Observe que isso é apenas uma alteração da taxa de aprendizado durante o processo de treinamento para lhe dar uma ideia de qual é uma taxa de aprendizado razoável e não deve ser confundido com a seleção da melhor taxa de aprendizado, o que é conhecido como [otimização de hiperparâmetros](https://www.tensorflow.org/tutorials/keras/keras_tuner).**

Para os otimizadores, você pode experimentar:
- [`tf.keras.optimizers.Adam`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)
- [`tf.keras.optimizers.SGD`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/SGD) com um momentum de 0.9

In [ ]:
def adjust_learning_rate():
    
    model = create_uncompiled_model()
    
    lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-6 * 10**(epoch / 20))
    
    ### INICIE SEU CÓDIGO AQUI
    
    # Selecione seu otimizador
    optimizer = None
    
    # Compile o modelo passando a perda apropriada
    model.compile(loss=None,
                  optimizer=optimizer, 
                  metrics=["mae"]) 
    
    ### TERMINE SEU CÓDIGO AQUI
    
    history = model.fit(dataset, epochs=100, callbacks=[lr_schedule])
    
    return history

In [ ]:
# Executar o treinamento com LR dinâmico
lr_history = adjust_learning_rate()

In [ ]:
# Traçar a perda para cada LR
plt.semilogx(lr_history.history["lr"], lr_history.history["loss"])
plt.axis([1e-6, 1, 0, 30])

## Compilação do modelo

Agora que você treinou o modelo variando a taxa de aprendizado, é hora de fazer o treinamento real que será usado para prever a série temporal. Para isso, conclua a função `create_model` abaixo.

Observe que você está reutilizando a arquitetura definida anteriormente em `create_uncompiled_model`. Agora você só precisa compilar esse modelo usando a perda, o otimizador (e a taxa de aprendizado) apropriados.

Dica:
- O treinamento deve ser muito rápido, portanto, se perceber que cada época está demorando mais do que alguns segundos, considere tentar uma arquitetura diferente.


- Se após a primeira época você obtiver uma saída como esta: `loss: nan - mae: nan`, é muito provável que sua rede esteja sofrendo com a explosão de gradientes. Esse é um problema comum se você usou o `SGD` como otimizador e definiu uma taxa de aprendizado muito alta. **Se você encontrar esse problema, considere reduzir a taxa de aprendizado ou usar o Adam com a taxa de aprendizado padrão.**

In [ ]:
def create_model():

    tf.random.set_seed(51)
    
    model = create_uncompiled_model()

    ### INICIE SEU CÓDIGO AQUI

    model.compile(loss=None,
                  optimizer=None,
                  metrics=["mae"])  
    
    ### TERMINE SEU CÓDIGO AQUI

    return model

In [ ]:
# Salvar uma instância do modelo
model = create_model()

# Treine-o
history = model.fit(dataset, epochs=50)

## Avaliação da previsão

Agora é hora de avaliar o desempenho da previsão. Para isso, você pode usar a função `compute_metrics` que codificou em uma tarefa anterior:

In [ ]:
def compute_metrics(true_series, forecast):
    
    mse = tf.keras.metrics.mean_squared_error(true_series, forecast).numpy()
    mae = tf.keras.metrics.mean_absolute_error(true_series, forecast).numpy()

    return mse, mae

Nesse momento, apenas o modelo que realizará a previsão está pronto, mas você ainda precisa calcular a previsão real. 

## Previsões de modelo mais rápidas

Na semana anterior, você usou um loop for para calcular as previsões para cada ponto da sequência. Essa abordagem é válida, mas há uma maneira mais eficiente de fazer a mesma coisa usando lotes de dados. O código para implementar isso é fornecido no `model_forecast` abaixo. Observe que o código é muito semelhante ao da função `windowed_dataset` com a diferença de que:

- O conjunto de dados é janelado usando `window_size` em vez de `window_size + 1`
- Não deve ser usado shuffle
- Não há necessidade de dividir os dados em recursos e rótulos
- Um modelo é usado para prever lotes do conjunto de dados

In [ ]:
def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.predict(ds)
    return forecast

In [ ]:
# Calcular a previsão para todas as séries
rnn_forecast = model_forecast(model, G.SERIES, G.WINDOW_SIZE).squeeze()

# Corte a previsão para obter apenas as previsões para o conjunto de validação
rnn_forecast = rnn_forecast[G.SPLIT_TIME - G.WINDOW_SIZE:-1]

# Plote-o
plt.figure(figsize=(10, 6))

plot_series(time_valid, series_valid)
plot_series(time_valid, rnn_forecast)

**Saída esperada:**

Uma série semelhante a esta:

<div>
<img src="images/expected.png" width="500"/>
</div>

In [ ]:
mse, mae = compute_metrics(series_valid, rnn_forecast)

print(f"mse: {mse:.2f}, mae: {mae:.2f} for forecast")

**Para ser aprovado nesta tarefa, sua previsão deve atingir um MAE de 4,5 ou menos.**

- Se a sua previsão não atingir esse limite, tente treinar novamente o modelo com uma arquitetura diferente (será necessário executar novamente as funções `create_uncompiled_model` e `create_model`) ou ajustar os parâmetros do otimizador.


- Se a sua previsão atingiu esse limite, execute a seguinte célula para salvar o seu modelo em um arquivo `tar` que será usado para avaliação e, depois disso, envie o seu trabalho para avaliação.


- Esse ambiente inclui um diretório fictício `SavedModel` que contém um modelo fictício treinado para uma época. **Para substituir esse arquivo pelo seu modelo real, você precisa executar a próxima célula antes de enviá-lo para avaliação.**


- Diferentemente da atividade avaliativa anterior, desta vez o modelo é salvo usando o formato `SavedModel`. Isso é feito porque o formato HDF5 não suporta totalmente as camadas `Lambda`.

In [ ]:
# Salve seu modelo no formato SavedModel
model.save('saved_model/my_model')

# Comprimir o diretório usando tar
! tar -czvf saved_model.tar.gz saved_model/

**Parabéns por ter concluído essa atividade!**

Você implementou com sucesso uma rede neural capaz de prever séries temporais aproveitando as camadas do Tensorflow para modelagem de sequências, como `RNNs` e `LSTMs`! **Isso resultou em uma previsão que corresponde (ou até mesmo supera) a da atividade avaliativa anterior durante o treinamento de metade das épocas.**

**Continue assim!**